In [ ]:
import requests

url = 'https://raw.githubusercontent.com/EinarOlafsson/spacr/main/spacr_simulation.py'
response = requests.get(url)
with open('spacr_simulation.py', 'w') as file:
    file.write(response.text)
    
import spacr_simulation as spacr

In [ ]:
#lopp over most variables
avg_genes_per_well_ls = [10] #5 [5]
avg_cells_per_well_ls = [100] #4 [200]
positive_mean_ls = [0.8] #4 [0.8]
avg_reads_per_gene_ls = [1000] #2 [1000]
sequencing_error_ls = [0.02] #3 [0.01]
well_ineq_coeff_ls = [0.3] #4 [0.2] lower mean more inequality
gene_ineq_coeff_ls = [0.8] #4 [0.2]
nr_plates_ls = [8] 
number_of_genes_ls = [1384] # [2000,4000,8000] 
number_of_active_genes_ls = [8] #[10]
replicates = 1

settings = {
    'name':'plates_2_4_8',
    'variable':'all',
    'src': '/media/olafsson/Data2/simulation2',
    'plot': True,
    'random_seed': False,
    'nr_plates': nr_plates_ls,
    'number_of_genes': number_of_genes_ls,
    'number_of_active_genes': number_of_active_genes_ls,
    'number_of_control_genes': 30,
    'avg_genes_per_well': avg_genes_per_well_ls,
    'avg_cells_per_well': avg_cells_per_well_ls,
    'positive_mean': positive_mean_ls,
    'avg_reads_per_gene': avg_reads_per_gene_ls,
    'sequencing_error': sequencing_error_ls,
    'well_ineq_coeff': well_ineq_coeff_ls,
    'gene_ineq_coeff': gene_ineq_coeff_ls,
    'max_workers': 1}

sim_ls = generate_paramiters(settings)
max_workers = settings['max_workers'] or cpu_count() - 4
with Manager() as manager:
    time_ls = manager.list()
    total_sims = len(sim_ls)
    with Pool(max_workers) as pool:
        result = pool.starmap_async(spacr.run_and_save, [(index, settings, time_ls, total_sims) for index, settings in enumerate(sim_ls)])
        while not result.ready():
            sleep(0.01)
            sims_processed = len(time_ls)
            average_time = np.mean(time_ls) if len(time_ls) > 0 else 0
            time_left = (((total_sims-sims_processed)*average_time)/max_workers)/60
            print(f'Progress: {sims_processed}/{total_sims} Time/simulation {average_time:.3f}sec Time Remaining {time_left:.3f} min.', end='\r', flush=True)
        result.get()